YouTube Comments dataset

### Imports

In [50]:
!pip install torchtext

You should consider upgrading via the '/home/ckm/python3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [51]:
import os
import math
import numpy as np
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from time import sleep


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

In [52]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [53]:
!nvidia-smi

Mon Dec 27 15:10:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.00    Driver Version: 470.82.00    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:17:00.0 Off |                    0 |
| N/A   79C    P0    74W / 250W |  11446MiB / 16160MiB |     79%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Load embedding

In [54]:
datapath = "/home/ckm/ck_project/codemix/code/selfnet_mihir/"
df_embedding = pd.read_csv(datapath+'hing_emb', sep=" ", quoting=3, header=None, index_col=0,skiprows=1)

In [55]:
df_embedding.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
nahi,-0.079912,0.522077,0.009111,-0.103475,-0.042439,0.229031,0.360524,0.047913,0.129397,0.184819,...,0.049488,-0.291750,0.188353,-0.099099,0.271938,0.311072,-0.446256,-0.093532,0.118786,-0.221102
to,0.268486,0.454443,-0.211620,-0.029896,-0.123828,0.119760,0.146776,0.152356,0.136670,-0.086144,...,0.256750,0.170485,-0.081619,-0.110159,-0.012980,0.211806,-0.197437,0.094797,-0.070250,-0.186570
ki,-0.342798,0.568509,0.049298,-0.111945,-0.312412,0.490620,0.442846,0.053863,0.177461,0.111002,...,0.491238,-0.074531,0.165319,-0.015633,0.151389,0.332968,-0.311262,0.057184,-0.281329,-0.346765
ke,-0.299528,0.510413,-0.033908,-0.064933,-0.439092,0.359395,0.342511,-0.102557,0.354761,0.632507,...,0.202599,-0.206487,0.178269,-0.108783,0.322591,0.347032,-0.558663,-0.018923,0.116121,-0.325165
se,-0.435697,0.357603,-0.040995,0.107330,-0.313671,0.268082,0.246396,0.005616,0.089185,0.249072,...,0.409665,-0.106801,-0.285788,0.028810,0.064475,0.117800,-0.666445,-0.167434,-0.310887,-0.367422


### Create vocab dictionary

In [56]:
m = df_embedding.to_numpy()
m.shape

(40599, 100)

In [57]:
embedding_matrix = df_embedding.to_numpy()

vocab = []

for word in list(df_embedding.index):
  vocab.append(str(word))

vocab_size , vocab_dim = embedding_matrix.shape
vocab_size, vocab_dim

(40599, 100)

In [58]:
len(vocab)

40599

In [59]:
word2idx = {w: idx for (idx, w) in enumerate(vocab)}
idx2word = {idx: w for (idx, w) in enumerate(vocab)}

In [60]:
len(word2idx) , len(idx2word)

(40598, 40599)

In [61]:
import itertools
dict(itertools.islice(word2idx.items(), 10))

{'nahi': 0,
 'to': 1,
 'ki': 2,
 'ke': 3,
 'se': 4,
 'ko': 5,
 'the': 6,
 'ka': 7,
 'me': 8,
 'bhi': 9}

### Read data

In [62]:
df = pd.read_csv('/home/ckm/ck_project/codemix/code/selfnet_mihir/Hindi_English_SAIL.csv')
df= df.sample(frac = 1)

train_df = df[:9500]
val_df = df[9500:10500]
test_df = df[10500:]

train_df.head()


train_data =  train_df['Sentence'].values.astype(str)
train_labels = train_df['Label'].values
train_labels = [int(label) for label in train_labels]

val_data =  val_df['Sentence'].values.astype(str)
val_labels = val_df['Label'].values
val_labels = [int(label) for label in val_labels]

test_data =  test_df['Sentence'].values.astype(str)
test_labels = test_df['Label'].values
test_labels = [int(label) for label in test_labels]

print(len(train_data), len(train_labels))
print(len(val_data), len(val_labels))
print(len(test_data), len(test_labels))



9500 9500
1000 1000
2101 2101


In [63]:
df.head()

,Sentence,Label
80,garbage bin bhai batao na mera comment pahla h...,2
2859,abhe yeh really salman ki id h kya,2
508,its not guddu's fault . . . . . prestige ka hi...,0
11227,Apne dil jeet lia bhai...Allah bless you,0
527,"my brother used to do these thngs for me , , h...",2


In [64]:

# post = pd.read_csv("positive_add1and2.csv",sep=',')
# neg = pd.read_csv("negative_add_1and2.csv",sep=',')

# data = []
# labels = []

# for index, row in post.iterrows():
#   if index >= 30000:
#     break
  
#   comment = str(row['tweet'])
#   data.append(comment)
#   labels.append(1)


# for index, row in neg.iterrows():
#   if index >= 30000:
#     break
  
#   comment = str(row['tweet'])
#   data.append(comment)
#   labels.append(0)
  

# len(data), len(labels)

# data, labels  = shuffle(data, labels, random_state = 40)

# data[0], labels[0]

### Tokenizer

In [65]:
tokenizer = get_tokenizer('basic_english')


def tokenized_tensor(data):

  output_tokenized = []

  for sentence in data:
    output = []
    tokenized = tokenizer(sentence)
    
    for word in tokenized:
      if word in word2idx:
        id = word2idx[word]
        output.append(id)
      else:
        word2idx[word] = len(word2idx)
        id = word2idx[word]
        output.append(id)

    output = torch.tensor(output)


    output_tokenized.append(output)

  return output_tokenized

In [66]:
# tokenized_sequences = tokenized_tensor(data)

train_tokenized_sequences = tokenized_tensor(train_data)

test_tokenized_seuqences = tokenized_tensor(test_data)

val_tokenized_seuquences = tokenized_tensor(val_data)


In [67]:
print(train_tokenized_sequences[1])

tensor([17297,   110,   455,  5316,     2,  3563,    28,    24])


In [68]:
word2idx['<PAD>'] = len(word2idx)
word2idx['<PAD>']

53012

In [69]:
len(word2idx)

53013

In [70]:
## Create embedding matrix

random_init = torch.nn.Parameter(torch.Tensor( (len(word2idx) - vocab_size), vocab_dim))
torch.nn.init.kaiming_uniform_(random_init, a=math.sqrt(5))


new_matrix = np.zeros( (len(word2idx), vocab_dim) )

new_matrix[:vocab_size, :] = embedding_matrix

embedding_matrix = new_matrix

embedding_matrix[vocab_size:, :] = random_init.detach().numpy()

In [71]:
embedding_matrix.shape

(53013, 100)

In [72]:
embedding_matrix.shape

(53013, 100)

In [73]:
# padded_sequences = pad_sequence(tokenized_sequences, batch_first= True, padding_value=107512)

train_padded_sequences = pad_sequence(train_tokenized_sequences, batch_first= True, padding_value=word2idx['<PAD>'])

val_padded_sequences = pad_sequence(val_tokenized_seuquences, batch_first= True, padding_value=word2idx['<PAD>'])

test_padded_sequences = pad_sequence(test_tokenized_seuqences, batch_first= True, padding_value=word2idx['<PAD>'])

In [74]:
(train_padded_sequences[1])

tensor([17297,   110,   455,  5316,     2,  3563,    28,    24, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
        53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 

### Dataset and Data loader 

In [75]:
class Dataset(torch.utils.data.Dataset):
    """
    This is our custom dataset class which will load the text and their corresponding labels into Pytorch tensors
    """
    def __init__(self, sequences, labels):
        self.labels = labels
        self.sequences = sequences

    def __getitem__(self, idx):
        sample = {}
        sequence = self.sequences[idx]
        label = torch.tensor(self.labels[idx])

        try:
            sample["label"] = label
            sample["token"] = sequence
        except Exception as e:
            print(e)
        
        return sample
    
    def __len__(self):
        return len(self.labels)
        

In [76]:
train_dataset = Dataset(train_padded_sequences, train_labels)

val_dataset = Dataset(val_padded_sequences, val_labels)

test_dataset = Dataset(test_padded_sequences, test_labels)

### Hyper parameters

In [77]:
## Hyper parameter

vocab_size = len(word2idx)
embed_dim = vocab_dim
seq_len = 192
hidden_size = 512
num_layer = 3
num_class = 3
batch_size = 32

LEARNING_RATE = 1e-3
EPOCHS = 30
CLIP = 0.3

In [78]:
# # Create datasets
# dataset = Dataset(padded_sequences, labels)

# split = 0.85
# train_size = int(split*len(dataset))
# val_size = len(dataset) - train_size

# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [79]:

## We call the dataloader class
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )

## For testing
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )


dataloaders = {'Train': train_loader, 'Val': val_loader}



In [80]:
len(train_loader), len(val_loader), len(test_loader)

(296, 31, 65)

In [81]:
max_len = -1
for comment in val_dataset:
  sentence = comment['token']
  max_len = max(max_len,sentence.shape[0])

print(max_len)

60


In [82]:
train_dataset[0]

{'label': tensor(2),
 'token': tensor([ 9964,  2901,    74,   859,    29,    39,  1110, 13667, 40598,    29,
         40599, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012, 53012,
         53012, 53012,

### Model

In [83]:
class SelfMatchingLayer(nn.Module):

    def __init__(self,  seq_length, embed_dim, **kwargs):

      super(SelfMatchingLayer, self).__init__()

      self.seq_length = seq_length
      self.embed_dim  = embed_dim

      self.P = torch.nn.Parameter(torch.Tensor(self.embed_dim, self.embed_dim))

      self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.P, a=math.sqrt(5))

      
    def forward(self, x):  
      
      # input shape: [batch, seq_len, embed_dim]


      #---------------------------------------------#
      # calculate weight vector a = {e_i . P.Q . e_j}
      #---------------------------------------------#

      out = torch.matmul(x,  self.P)   #out shape: [batch, seq_len, embed_dim]

      out = torch.matmul(out, torch.transpose(x, 1, 2))   #out shape: [batch, seq_len, seq_len]

      out = F.gelu(out)         # apply non linear activation

      #------------------------------------#
      # take row wise mean and apply softmax
      #------------------------------------#
      out = torch.mean(out, 2)  #out shape: [batch, seq_len, seq_len]

      out = torch.softmax(out, 0)     #out shape: [batch, seq_len, seq_len]

      out = out.unsqueeze(1)          #out shape: [batch, 1, seq_len]

      #-------------------------------------------#
      # calculate weighted embedding of every word
      #-------------------------------------------#
      out = torch.matmul(out, x)

      out = out.squeeze(1)

      return out      #out shape: [batch, seq_len]


In [84]:
word2idx['<PAD>']

53012

In [85]:
class SelfNet(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class):
        super(SelfNet, self).__init__()


        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = word2idx['<PAD>'])
        self.embedding.load_state_dict({'weight': torch.from_numpy(embedding_matrix)})
        self.embedding.weight.requires_grad = True

        self.selfnet_layer = SelfMatchingLayer(seq_len, embed_dim)

        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = num_layer, dropout = 0.3, bidirectional = True, batch_first = True )


        self.fc1 = nn.Linear(2* hidden_size + embed_dim , hidden_size//4)
        self.fc2 = nn.Linear(hidden_size//4, num_class)

        self.dropout = nn.Dropout(0.3)



    def forward(self, input):

        embedded = self.embedding(input)  #out shape = [batch, seq_len, embed_dim] 

        selfmatch_output = self.selfnet_layer(embedded)  #out shape = [batch, seq_len] 

        lstm_out, _ = self.lstm(embedded)     

        lstm_out = lstm_out[:, -1, :]      #out shape = [batch, 2 * hidden_size]      

        concat = torch.cat( (selfmatch_output, lstm_out), 1)     #out shape = [batch, 2 * hidden_size + seq_len ]      

        linear_out = self.dropout(F.relu(self.fc1(concat)))     #out shape = [batch, hidden_size]      

        final_out = self.fc2(linear_out)     #out shape = [batch, 2]      

        return final_out

In [86]:

### Test
model = SelfNet( vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class)
model = model.to(device)

# for batch in train_loader:
#   x = batch['token'].to(device)
#   out = model(x)
#   print(out.shape)
#   break



In [87]:
model

SelfNet(
  (embedding): Embedding(53013, 100, padding_idx=53012)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

### Optimizer and loss

In [88]:
#optimizer
optimizer = Adam(model.parameters(), lr = LEARNING_RATE, eps=1e-8)
#Loss function
criterion = nn.CrossEntropyLoss()

In [89]:
#to calculate accuracy

def get_accuracy(preds, labels):
  total_acc = 0.0
  
  for i in range(len(labels)):
    if labels[i] == preds[i]:
      total_acc+=1.0
  
  return total_acc / len(labels)

### Training

In [90]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.2, patience=5, threshold=0.0008, verbose = True)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5, verbose = True)


Adjusting learning rate of group 0 to 1.0000e-03.


In [91]:
PATH = '/home/ckm/ck_project/codemix/models/selfnet_mean+gelu_SAIL.pt'

In [92]:
best_valid_f1 = 0.0000

for epoch in range(0, EPOCHS):
  

    print('-'*50)
    print('Epoch {}/{}'.format(epoch+1, EPOCHS))

    for phase in ['Train', 'Val']:

        loss = 0.0   #epoch loss
        accuracy = 0.0   #epoch accuracy

        y_true = []
        y_pred = []

        if phase == 'Train':
            model.train()
        else:
            model.eval()
        
        with tqdm(dataloaders[phase], unit="batch") as tepoch:

          for batch in tepoch:
            labels = batch["label"].to(device)
            text = batch["token"].to(device)
            

            output = model(text)

            loss = criterion(output, labels)

            if phase == 'Train':

                #zero gradients
                optimizer.zero_grad() 

                # Backward pass  (calculates the gradients)
                loss.backward()   

                # gradient clipping
                nn.utils.clip_grad_norm_(model.parameters(), CLIP)    

                optimizer.step()             # Updates the weights    

            sleep(0.1)
            _, preds = output.data.max(1)
            y_pred.extend(preds.tolist())
            y_true.extend(labels.tolist())
            
            batch_acc = get_accuracy(preds.tolist(), labels.tolist())
            
            
            loss += loss.item()
            accuracy+= batch_acc

              
          epoch_loss = loss / (len(dataloaders[phase]))
          epoch_acc = accuracy / (len(dataloaders[phase]))

          print(phase + ":")
          
          
          #print(confusion_matrix(y_true, y_pred))
          pre = precision_score(y_true, y_pred, average='weighted')
          recall = recall_score(y_true, y_pred, average='weighted')
          f1 = f1_score(y_true, y_pred, average='weighted')
          

          print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))
          # save best model
          print()
          
            
          if phase == 'Val':
                
                if f1 > best_valid_f1:
                    best_valid_f1 = f1
                    
                    torch.save(model.state_dict(), PATH)
                    print('Model Saved!')
                    
                scheduler.step()
                    


--------------------------------------------------
Epoch 1/30


100%|██████████| 296/296 [04:30<00:00,  1.09batch/s]


Train:
F1: 0.3347, Precision: 0.4124, Recall : 0.4560, Accuracy: 0.4560, Loss: 0.0078.



100%|██████████| 31/31 [00:03<00:00,  8.14batch/s]


Val:
F1: 0.4323, Precision: 0.4869, Recall : 0.4335, Accuracy: 0.4335, Loss: 0.0692.

Model Saved!
Adjusting learning rate of group 0 to 5.0000e-04.
--------------------------------------------------
Epoch 2/30


...

Val:
F1: 0.4705, Precision: 0.5454, Recall : 0.4798, Accuracy: 0.4798, Loss: 0.0697.

Adjusting learning rate of group 0 to 1.8626e-12.
--------------------------------------------------
Epoch 30/30


100%|██████████| 296/296 [04:29<00:00,  1.10batch/s]


Train:
F1: 0.5687, Precision: 0.5939, Recall : 0.5913, Accuracy: 0.5913, Loss: 0.0066.



100%|██████████| 31/31 [00:03<00:00,  8.04batch/s]

Val:
F1: 0.4647, Precision: 0.5478, Recall : 0.4718, Accuracy: 0.4718, Loss: 0.0678.

Adjusting learning rate of group 0 to 9.3132e-13.


### Testing

In [93]:
model = SelfNet( vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class)

In [94]:
model.load_state_dict(torch.load(PATH))

model.to(device)


SelfNet(
  (embedding): Embedding(53013, 100, padding_idx=53012)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [95]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

100%|██████████| 65/65 [00:27<00:00,  2.34batch/s]


Inference:

[[207  51 410]
 [ 43 113 278]
 [ 77  63 838]]

F1: 0.5567, Precision: 0.5567, Recall : 0.5567, Accuracy: 1.1673, Loss: 0.0540.


In [96]:
model.load_state_dict(torch.load(PATH))

model.to(device)


SelfNet(
  (embedding): Embedding(53013, 100, padding_idx=53012)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [97]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

100%|██████████| 65/65 [00:26<00:00,  2.50batch/s]


Inference:

[[206  58 403]
 [ 41 116 281]
 [ 74  77 824]]

F1: 0.5157, Precision: 0.5592, Recall : 0.5510, Accuracy: 1.1552, Loss: 0.0619.


In [98]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

100%|██████████| 65/65 [00:28<00:00,  2.29batch/s]


Inference:

[[206  55 402]
 [ 45 120 270]
 [ 78  69 835]]

F1: 0.5582, Precision: 0.5582, Recall : 0.5582, Accuracy: 1.1704, Loss: 0.0624.
